### AUTHOR - ZEEL B PATEL

# Introduction to Neural Networks

## Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Convention

In [ ]:
n0 = 3
n1 = 2
layer = nn.Linear(n0, n1)
layer

Linear(in_features=3, out_features=2, bias=True)

In [ ]:
layer.weight.shape

torch.Size([2, 3])

In [ ]:
layer.bias.shape

torch.Size([2])

In [ ]:
for i in range(n0):
    print(layer.weight[0, i])

tensor(-0.5629, grad_fn=<SelectBackward0>)
tensor(-0.4873, grad_fn=<SelectBackward0>)
tensor(-0.1570, grad_fn=<SelectBackward0>)


### A 2-layer network

In [ ]:
mlp_2layers = nn.Sequential(
    nn.Linear(4, 3),
    nn.ReLU(),
    nn.Linear(3, 2),
    nn.ReLU(),
    nn.Linear(2, 1)
)

In [ ]:
params = dict(mlp_2layers.named_parameters())
{name: param.shape for name, param in params.items()}

{'0.weight': torch.Size([3, 4]),
 '0.bias': torch.Size([3]),
 '2.weight': torch.Size([2, 3]),
 '2.bias': torch.Size([2]),
 '4.weight': torch.Size([1, 2]),
 '4.bias': torch.Size([1])}

## Vectorization with `torch.vmap`

In [ ]:
!pip install jaxtyping
!pip install beartype

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.1 MB/s eta 0:00:00


In [ ]:
from jaxtyping import Float
from torch import Tensor
from beartype import beartype

### Quick intro to jaxtyping

#### Scalars

In [ ]:
scalar_type = Float[Tensor, ""]

In [ ]:
scalar = torch.tensor(1.0)
scalar.shape

torch.Size([])

In [ ]:
isinstance(scalar, scalar_type)

True

In [ ]:
non_scalar = torch.tensor([1.0])
non_scalar.shape

torch.Size([1])

In [ ]:
isinstance(non_scalar, scalar_type)

False

#### Vectors

In [ ]:
vector_type = Float[Tensor, "n0"]

In [ ]:
isinstance(non_scalar, vector_type)

True

In [ ]:
vector = torch.tensor([1.0, 2.0])
vector.shape

torch.Size([2])

In [ ]:
isinstance(vector, vector_type)

True

#### Matrices

In [ ]:
matrix_type = Float[Tensor, "n0 n1"]

In [ ]:
isinstance(vector, matrix_type)

False

In [ ]:
matrix = torch.tensor([[1.0, 2.0]])
matrix.shape

torch.Size([1, 2])

In [ ]:
isinstance(matrix, matrix_type)

True

In [ ]:
another_matrix = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
another_matrix.shape

torch.Size([2, 3])

In [ ]:
isinstance(another_matrix, matrix_type)

True

#### Tensors

In [ ]:
tensor_type = Float[Tensor, "n0 n1 n2"]

In [ ]:
tensor = torch.tensor([[[1.0, 2.0], [3.0, 4.0]], [[5.0, 6.0], [7.0, 8.0]]])
tensor.shape

torch.Size([2, 2, 2])

In [ ]:
isinstance(tensor, tensor_type)

True

### Quick intro to beartype

In [ ]:
def call_my_name(name):
    return f"Hello, {name}!"

In [ ]:
call_my_name("John")

'Hello, John!'

In [ ]:
call_my_name(123)

'Hello, 123!'

In [ ]:
@beartype
def secured_call_my_name(name: str) -> str:
    return f"Hello, {name}!"

In [ ]:
secured_call_my_name("John")

'Hello, John!'

In [ ]:
secured_call_my_name(123)

BeartypeCallHintParamViolation: Function __main__.secured_call_my_name() parameter name=123 violates type hint <class 'str'>, as int 123 not instance of str.

### Vectorization

On which dimensions should we apply the vectorization?
- Current layer's neurons size
- Number of examples

In [ ]:
n = 50
a = torch.rand(n, n0)
a.shape

torch.Size([50, 3])

In [ ]:
activation = F.relu

@beartype
def forward(a: Float[Tensor, "n0"], w: Float[Tensor, "n0"], b: Float[Tensor, ""]) -> Float[Tensor, ""]:
    z = (a * w).sum() + b  # () * () + () -> ()
    a = activation(z)  # () -> ()
    return a  # ()

In [ ]:
dummy_a = torch.rand(n0)
dummy_w = torch.rand(n0)
dummy_b = torch.rand(())
print(dummy_a.shape, dummy_w.shape, dummy_b.shape)

torch.Size([3]) torch.Size([3]) torch.Size([])


In [ ]:
forward(dummy_a, dummy_w, dummy_b).shape

torch.Size([])

In [ ]:
forward(a[0], layer.weight[0], layer.bias[0])

tensor(0., grad_fn=<ReluBackward0>)

#### Vectorization over current layer's neurons size

| input | shape in forward | shape in vectorized forward |
| --- | --- | --- |
| a | [n0=3] | [n0=3] |
| w | [n0=3] | [n1=2, n0=3] |
| b | [] | [n1=2] |
| output | [] | [n1=2] |

In [ ]:
v1_forward = torch.vmap(forward, in_dims=(None, 0, 0), out_dims=0)

In [ ]:
layer.weight.shape

torch.Size([2, 3])

In [ ]:
layer.bias.shape

torch.Size([2])

In [ ]:
out = v1_forward(a[0], layer.weight, layer.bias)
out.shape

torch.Size([2])

#### Vectorization over number of examples

| input | shape in forward | shape in vectorized forward |
| --- | --- | --- |
| a | [n0=3] | [n=50, n0=3] |
| w | [n1=2, n0=3] | [n1=2, n0=3] |
| b | [n1=2] | [n1=2] |
| output | [n1=2] | [n=50, n1=2] |

In [ ]:
v2_forward = torch.vmap(v1_forward, in_dims=(0, None, None), out_dims=0)

In [ ]:
final_out = v2_forward(a, layer.weight, layer.bias)
final_out.shape

torch.Size([50, 2])

#### Comparing with torch model forward pass

In [ ]:
layer_out = F.relu(layer(a))
torch.allclose(final_out, layer_out)

True

## XOR example

### Define inputs, outputs, weights and biases

In [ ]:
xor_x = torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
print(xor_x)
print(xor_x.shape)

tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]])
torch.Size([4, 2])


In [ ]:
xor_y = torch.tensor([[0.0], [1.0], [1.0], [0.0]])
print(xor_y)
print(xor_y.shape)

tensor([[0.],
        [1.],
        [1.],
        [0.]])
torch.Size([4, 1])


In [ ]:
W_1 = torch.tensor([[1.0, 1.0], [1.0, 1.0]])
b_1 = torch.tensor([0.0, -1.0])
print(W_1)
print(b_1)
print(W_1.shape, b_1.shape)

tensor([[1., 1.],
        [1., 1.]])
tensor([ 0., -1.])
torch.Size([2, 2]) torch.Size([2])


In [ ]:
W_2 = torch.tensor([[1.0, -2.0]])
b_2 = torch.tensor([0.0])
print(W_2)
print(b_2)
print(W_2.shape, b_2.shape)

tensor([[ 1., -2.]])
tensor([0.])
torch.Size([1, 2]) torch.Size([1])


### Forward pass

In [ ]:
a1 = v2_forward(xor_x, W_1, b_1)
a1

tensor([[0., 0.],
        [1., 0.],
        [1., 0.],
        [2., 1.]])

In [ ]:
a2 = v2_forward(a1, W_2, b_2)
a2

tensor([[0.],
        [1.],
        [1.],
        [0.]])

### Forward pass with torch neural network

In [ ]:
model = nn.Sequential(nn.Linear(2, 2), nn.ReLU(), nn.Linear(2, 1))
print(model[0])
print(model[1])
print(model[2])

Linear(in_features=2, out_features=2, bias=True)
ReLU()
Linear(in_features=2, out_features=1, bias=True)


In [ ]:
model[0].weight.data = W_1
model[0].bias.data = b_1
model[2].weight.data = W_2
model[2].bias.data = b_2

In [ ]:
model_out = model(xor_x)
model_out

tensor([[0.],
        [1.],
        [1.],
        [0.]], grad_fn=<AddmmBackward0>)